In [51]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import requests
import pandas as pd

import zipfile
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import seaborn as sns

import dart_fss as dart_fss

kospi_url = "http://data-dbg.krx.co.kr/svc/apis/idx/kospi_dd_trd" 
stock_url = "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd" 
krx_api_key = "956087495A1D4769A88B0F3411D0890EF02445EB"

dart_api_key = '0728a6d81b70b86763fd38daafcc1fccff0beedb'
code_url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={dart_api_key}'
dart_url = 'https://opendart.fss.or.kr/api/fnlttSinglIndx.json'

In [114]:
def add_days(date_str: str, days: int) -> str:
    date = datetime.strptime(date_str, "%Y%m%d")
    new_date = date + timedelta(days=days)
    return new_date.strftime("%Y%m%d")

def get_closing_price_for_date(date: str, stock_name: str) -> float:
    url = stock_url
    headers = {
        'AUTH_KEY': krx_api_key,
    }

    params = {
        'basDd': date,
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if 'OutBlock_1' in data and data['OutBlock_1']:
            df_temp = pd.DataFrame(data['OutBlock_1'])[['ISU_NM', 'TDD_CLSPRC']]
            df_filtered = df_temp[df_temp['ISU_NM'] == stock_name]
            if not df_filtered.empty:
                return pd.to_numeric(df_filtered['TDD_CLSPRC'].iloc[0], errors='coerce')
    else:
        print(f"Error {response.status_code}: {response.text}")
    
    return None

def get_stock_data(start_date: str, end_date: str, stock_name: str) -> pd.DataFrame:
    url = stock_url
    headers = {
        'AUTH_KEY': krx_api_key,
    }

    df_stock = pd.DataFrame()

    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        bas_d = current_date.strftime("%Y%m%d")
        params = {
            'basDd': bas_d,
        }
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if 'OutBlock_1' in data and data['OutBlock_1']:
                df_temp = pd.DataFrame(data['OutBlock_1'])[['BAS_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'ACC_TRDVOL']]
                df_filtered = df_temp[df_temp['ISU_NM'] == stock_name]
                df_stock = pd.concat([df_stock, df_filtered], ignore_index=True)
        else:
            print(f"Error {response.status_code}: {response.text}")

        current_date += timedelta(days=1)
        
    
    # 데이터 타입 변환
    df_stock['TDD_CLSPRC'] = pd.to_numeric(df_stock['TDD_CLSPRC'], errors='coerce')
    df_stock['TDD_HGPRC'] = pd.to_numeric(df_stock['TDD_HGPRC'], errors='coerce')
    df_stock['TDD_LWPRC'] = pd.to_numeric(df_stock['TDD_LWPRC'], errors='coerce')
    df_stock['ACC_TRDVOL'] = pd.to_numeric(df_stock['ACC_TRDVOL'], errors='coerce')
    return df_stock

def get_report_code(date_str: str) -> str:
    # 입력된 날짜를 datetime 객체로 변환
    date = datetime.strptime(date_str, "%Y%m%d")
    month = date.month

    # 보고서 코드 결정
    if month in [1, 2, 3]:
        return "11013"
    elif month in [4, 5, 6]:
        return "11012"
    elif month in [7, 8, 9]:
        return "11014"
    else:
        return "11011"

def get_corp_code(company_name: str):
    stock_name = company_name

    # 기업 고유번호 XML 파일 다운로드 URL
    url = code_url

    # XML 파일 다운로드
    response = requests.get(url)
    zip_path = 'corpCode.zip'

    with open(zip_path, 'wb') as file:
        file.write(response.content)

    # ZIP 파일 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

    # XML 파일 파싱
    xml_file = 'CORPCODE.xml'
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 삼성전자의 고유번호 찾기
    corp_name = stock_name
    corp_code = None

    for company in root.findall('list'):
        if company.find('corp_name').text == corp_name:
            corp_code = company.find('corp_code').text
            break

    os.remove(zip_path)
    os.remove(xml_file)
    
    return corp_code
    
def get_financial_data(date_str: str, company_name: str) -> pd.DataFrame:
    # 각 지표 유형에 따른 코드 설정
    category_codes = {
        '수익성지표': 'M210000',
        '안정성지표': 'M220000',
        '성장성지표': 'M230000',
        '활동성지표': 'M240000'
    }
    df_result = pd.DataFrame()
    
    params = {
        'crtfc_key': dart_api_key,
        'corp_code': get_corp_code(company_name),
        'bsns_year': date_str[:4],
        'reprt_code': get_report_code(date_str),
        'idx_cl_code': 'M210000'
    }
    print(params)
    # API 요청
    response = requests.get(dart_url, params=params)
    data = response.json()

    # 응답 상태 확인
    if data.get('status') == '000':  # 성공 상태 코드
        index_class = pd.DataFrame(data['list'])
        filtered_df = index_class[index_class['idx_nm'].isin(['세전계속사업이익률', 'ROE'])]
        df_result = pd.concat([df_result, filtered_df], ignore_index=True)
    else:
        print("API 오류:", data.get('message'))
        
    params = {
        'crtfc_key': dart_api_key,
        'corp_code': get_corp_code(company_name),
        'bsns_year': date_str[:4],
        'reprt_code': get_report_code(date_str),
        'idx_cl_code': 'M230000'
    }
    # API 요청
    response = requests.get(dart_url, params=params)
    data = response.json()

    # 응답 상태 확인
    if data.get('status') == '000':  # 성공 상태 코드
        index_class = pd.DataFrame(data['list'])
        filtered_df = index_class[index_class['idx_nm'].isin(['매출액증가율(YoY)'])]
        df_result = pd.concat([df_result, filtered_df], ignore_index=True)
    else:
        print("API 오류:", data.get('message'))
    return df_result

def add_technical_indicators(df_stock: pd.DataFrame, specific_date: str = None) -> pd.DataFrame:
    # 이동평균선 생성
    df_stock['5일이동평균'] = df_stock['TDD_CLSPRC'].rolling(window=5).mean()
    df_stock['20일이동평균'] = df_stock['TDD_CLSPRC'].rolling(window=20).mean()
    df_stock['50일이동평균'] = df_stock['TDD_CLSPRC'].rolling(window=50).mean()

    # RSI 생성 (14일 기준)
    delta = df_stock['TDD_CLSPRC'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df_stock['RSI'] = 100 - (100 / (1 + rs))

    # 지지선 및 저항선 설정
    df_stock['지지선'] = df_stock['TDD_LWPRC'].rolling(window=20).min()
    df_stock['저항선'] = df_stock['TDD_HGPRC'].rolling(window=20).max()

    # 거래량 분석 (20일 이동평균)
    df_stock['20일거래량이동평균'] = df_stock['ACC_TRDVOL'].rolling(window=20).mean()

    # 볼린저 밴드 생성 (20일 이동평균과 표준편차 이용)
    df_stock['20일표준편차'] = df_stock['TDD_CLSPRC'].rolling(window=20).std()
    df_stock['볼린저상한'] = df_stock['20일이동평균'] + (2 * df_stock['20일표준편차'])
    df_stock['볼린저하한'] = df_stock['20일이동평균'] - (2 * df_stock['20일표준편차'])

    # 특정 날짜에 맞는 데이터만 반환 (선택사항)
    if specific_date:
        df_stock = df_stock[df_stock['BAS_DD'] == specific_date]
    
    return df_stock

def predict_future_price(data, current_price, financials=None):
    up_ratio = 1.0
    down_ratio = 1.0

    if data['5일이동평균'].iloc[-1] > data['20일이동평균'].iloc[-1] and data['20일이동평균'].iloc[-1] > data['50일이동평균'].iloc[-1]:
        moving_avg = '강한 상승 추세'
        up_ratio += 0.10  # 강한 상승 추세라면 상승 비율에 10% 추가
    elif data['5일이동평균'].iloc[-1] < data['20일이동평균'].iloc[-1] and data['20일이동평균'].iloc[-1] < data['50일이동평균'].iloc[-1]:
        moving_avg = '강한 하락 추세'
        down_ratio += 0.10  # 강한 하락 추세라면 하락 비율에 10% 추가
    elif data['5일이동평균'].iloc[-1] > data['20일이동평균'].iloc[-1]:
        moving_avg = '단기 상승 추세'
        up_ratio += 0.07  # 단기 상승 추세라면 상승 비율에 7% 추가
    elif data['5일이동평균'].iloc[-1] < data['20일이동평균'].iloc[-1]:
        moving_avg = '단기 하락 추세'
        down_ratio += 0.07  # 단기 하락 추세라면 하락 비율에 7% 추가
    else:
        moving_avg = '횡보 추세'

    # RSI 분석
    if data['RSI'].iloc[-1] > 70:
        rsi = '과매수'
        down_ratio += 0.06  # 과매수 상태라면 하락 비율에 6% 추가
    elif data['RSI'].iloc[-1] < 30:
        rsi = '과매도'
        up_ratio += 0.06  # 과매도 상태라면 상승 비율에 6% 추가
    else:
        rsi = '중립'

    # 볼린저 밴드 분석
    if data['TDD_CLSPRC'].iloc[-1] > data['볼린저상한'].iloc[-1]:
        bollinger_band = '상단 돌파'
        down_ratio += 0.04  # 상단 돌파라면 하락 비율에 4% 추가
    elif data['TDD_CLSPRC'].iloc[-1] < data['볼린저하한'].iloc[-1]:
        bollinger_band = '하단 돌파'
        up_ratio += 0.04  # 하단 돌파라면 상승 비율에 4% 추가
    else:
        bollinger_band = '안정적'

    # 재무 지표 분석
#    if financials == '양호':
#        up_ratio += 0.12  # 재무 상태가 양호하다면 상승 비율에 12% 추가

    # 최종 예측 계산
#    if up_ratio > down_ratio:
#        future_price = current_price * up_ratio
#    elif down_ratio > up_ratio:
#        future_price = current_price / down_ratio  # 하락 비율을 고려하여 가격 감소 계산 (0보다 크게 유지)
#    else:
#        future_price = current_price  # 상승과 하락 비율이 같으면 현재 가격 유지

    future_price = current_price * (up_ratio / down_ratio)
    return future_price
        
    print(up_ratio, down_ratio)

    return future_price

In [113]:
def main():
    end_date = '20241101'
    stock_names = ['삼성전자', 'SK하이닉스', '현대차', 'LG에너지솔루션', '삼성바이오로직스', 'NAVER']  # 여러 회사명을 리스트로 저장
    krx_api_key = 'YOUR_KRX_API_KEY'
    
    for stock_name in stock_names:
        stock_data = get_stock_data(add_days(end_date, -100), end_date, stock_name)
        stock_data_with_indicators = add_technical_indicators(stock_data, end_date)
        
        price = predict_future_price(stock_data_with_indicators, pd.to_numeric(stock_data_with_indicators['TDD_CLSPRC'], errors='coerce').iloc[-1], None)
        
        dd = add_days(end_date, 90)
        print(end_date[:4] + '년' + end_date[4:6] + '월' + end_date[6:] + '일 ' + stock_name + ' 주식 가격 예측 : ' + str(pd.to_numeric(stock_data_with_indicators['TDD_CLSPRC'], errors='coerce').iloc[-1]))
        print(dd[:4] + '년' + dd[4:6] + '월' + dd[6:] + '일 ' + stock_name + ' 주식 가격 예측 : ' + str(price))

if __name__ == "__main__":
    main()


2024년11월01일 삼성전자 주식 가격 예측 : 58300
2025년01월30일 삼성전자 주식 가격 예측 : 53000.0
2024년11월01일 SK하이닉스 주식 가격 예측 : 182200
2025년01월30일 SK하이닉스 주식 가격 예측 : 200420.00000000003
2024년11월01일 현대차 주식 가격 예측 : 213500
2025년01월30일 현대차 주식 가격 예측 : 205736.36363636365
2024년11월01일 LG에너지솔루션 주식 가격 예측 : 412500
2025년01월30일 LG에너지솔루션 주식 가격 예측 : 453750.00000000006
2024년11월01일 삼성바이오로직스 주식 가격 예측 : 998000
2025년01월30일 삼성바이오로직스 주식 가격 예측 : 932710.2803738317
2024년11월01일 NAVER 주식 가격 예측 : 169700
2025년01월30일 NAVER 주식 가격 예측 : 158598.13084112148


In [118]:
def main():
    end_date = '20240502'
    stock_names = ['삼성전자', 'SK하이닉스', '현대차', 'LG에너지솔루션', '삼성바이오로직스', 'NAVER']  # 여러 회사명을 리스트로 저장
    krx_api_key = 'YOUR_KRX_API_KEY'
    
    for stock_name in stock_names:
        stock_data = get_stock_data(add_days(end_date, -100), end_date, stock_name)
        stock_data_with_indicators = add_technical_indicators(stock_data, end_date)
        sd = get_closing_price_for_date(add_days(end_date, 90), stock_name)
        
        price = predict_future_price(stock_data_with_indicators, pd.to_numeric(stock_data_with_indicators['TDD_CLSPRC'], errors='coerce').iloc[-1], None)
        
        dd = add_days(end_date, 90)
        print(end_date[:4] + '년' + end_date[4:6] + '월' + end_date[6:] + '일 ' + stock_name + ' 주식 가격 예측 : ' + str(pd.to_numeric(stock_data_with_indicators['TDD_CLSPRC'], errors='coerce').iloc[-1]))
        print(dd[:4] + '년' + dd[4:6] + '월' + dd[6:] + '일 ' + stock_name + ' 주식 가격 예측 : ' + str(price))
        print(dd[:4] + '년' + dd[4:6] + '월' + dd[6:] + '일 ' + stock_name + ' 실제 주식 가격 : ' + str(sd))

if __name__ == "__main__":
    main()


2024년05월02일 삼성전자 주식 가격 예측 : 78000
2024년07월31일 삼성전자 주식 가격 예측 : 72897.19626168224
2024년07월31일 삼성전자 실제 주식 가격 : 83900
2024년05월02일 SK하이닉스 주식 가격 예측 : 173600
2024년07월31일 SK하이닉스 주식 가격 예측 : 162242.9906542056
2024년07월31일 SK하이닉스 실제 주식 가격 : 194600
2024년05월02일 현대차 주식 가격 예측 : 249000
2024년07월31일 현대차 주식 가격 예측 : 266430.0
2024년07월31일 현대차 실제 주식 가격 : 249000
2024년05월02일 LG에너지솔루션 주식 가격 예측 : 389000
2024년07월31일 LG에너지솔루션 주식 가격 예측 : 416230.0
2024년07월31일 LG에너지솔루션 실제 주식 가격 : 324000
2024년05월02일 삼성바이오로직스 주식 가격 예측 : 775000
2024년07월31일 삼성바이오로직스 주식 가격 예측 : 704545.4545454545
2024년07월31일 삼성바이오로직스 실제 주식 가격 : 938000
2024년05월02일 NAVER 주식 가격 예측 : 188800
2024년07월31일 NAVER 주식 가격 예측 : 171636.36363636362
2024년07월31일 NAVER 실제 주식 가격 : 174000
